In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 3 / Assignment 1*
# Convolutional Neural Networks (CNNs)

# Assignment

케라스를 이용한 바이너리 이미지 분류 모델에 3가지 CNN 모델을 적용하여 보는 과제입니다. <br/>

- [데이터 다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/mountainForest.zip)

산의 이미지(./data/mountin/*)와 숲의 이미지(./data/forest/*)를 분류하는 문제입니다. <br/>
산을 Positive (1)로, 숲 이미지를 Negative(0)로 레이블링 하여줍니다.

클래스당 약 350개의 이미지로 이루어져 있는데요.<br/>
표본이 작다는 점을 감안하면 현실적으로 어려운 문제입니다.

하지만 이번 과제에서는 해당 데이터에 여러 가지 모델을 적용해보는는 것에 중점을 두어 봅시다. <br/>
과제를 통해 이미지 분류에 적용할 수 있는 여러 모델을 알아보고 서로를 비교하는 데 익숙해져 보면 좋겠죠?

# Code

## Part 1 : Pre-trained Model



Keras에서 제공하는 pre-trained 모델인 ResNet50을 불러와서 사용해봅니다. [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1)은 50 개의 layer를 가진  CNN기반의 모델입니다. <br/>
이미지를 [1000 개의 클래스로](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt)를 분류하는 모델인데요. 우리가 풀어야 할 과제는 2가지 이므로 마지막 출력단을 변경해서 사용해 볼 수 있습니다.


`ResNet50`을 불러올 때, **`include_top=False`** 로 하면, 기존 1000가지 클래스로의 분류 문제를 풀 수 있는 ResNet 모델에서 Fully Connected layer 부분을 제거해주는 역할을 합니다.

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model

resnet = ResNet50(weights='imagenet', include_top=False)
```

아래 부분은 ResNet50 레이어들의 파라미터를 학습하지 않도록 설정합니다. <br/>
이렇게 설정된 매개 변수는 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트 되지 않습니다.

```python
for layer in resnet.layers:
    layer.trainable = False
```

모델에 추가로 **`Fully-conneted layer(Dense)`** 를 추가해야 합니다. <br/>
사전 학습 모델을 불러오면서 최상위 레이어인 **`Fully-conneted layer`** 를 제거했기 때문이지요.

새로 추가하는 **`Fully-conneted layer`** 에서는 목적인 이진 분류에 맞게 출력층을 설계하여 주어야 합니다. <br/> **`GlobalAveragePooling2D`** 레이어는 마지막 컨벌루션 레이어 출력(2 차원) 각각의 평균을 취해주어 **`Dense`** 층에 들어갈 수 있도록 해줍니다.

```python
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # 출력층을 설계합니다.
model = Model(resnet.input, predictions)
```

### Load in Data

[Keras ImageDataGenerator](https://keras.io/api/preprocessing/image/) 를 참고하여 데이터를 불러옵니다. <br/>
위 링크뿐만 아니라 구글링을 통해 ImageDataGenerator 라이브러리에 대한 여러 예제를 조사하고 참고해보세요. 

Notebook을 여러분의 Google Drive에 Mount 한 후에 이미지를 불러오도록 합니다.

In [30]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [31]:
import os

base_dir = '/content/drive/MyDrive/Colab Notebooks/RazielData/mountainForest'

train_dir = os.path.join(base_dir, 'train')
train_forest_dir = os.path.join(train_dir, 'forest')
train_mountain_dir = os.path.join(train_dir, 'mountain')

validation_dir = os.path.join(base_dir, 'validation')
validation_forest_dir = os.path.join(validation_dir, 'forest')
validation_mountain_dir = os.path.join(validation_dir, 'mountain')

In [32]:
train_forest_fnames = os.listdir(train_forest_dir)
train_forest_fnames.sort()
print(train_forest_fnames[:10])

train_mountain_fnames = os.listdir(train_mountain_dir)
train_mountain_fnames.sort()
print(train_mountain_fnames[:10])

['.DS_Store', '.ipynb_checkpoints', 'art114.jpg', 'for102.jpg', 'for105.jpg', 'for106.jpg', 'for110.jpg', 'for112.jpg', 'for114.jpg', 'for116.jpg']
['.DS_Store', '.ipynb_checkpoints', 'art1131.jpg', 'art1132.jpg', 'gre242.jpg', 'land10.jpg', 'land11.jpg', 'land13.jpg', 'land130.jpg', 'land131.jpg']


In [33]:
from tensorflow.keras import layers

In [34]:
img_input = layers.Input(shape=(224*224*3))

In [35]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Found 533 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [36]:
for layer in resnet.layers:
    layer.trainable = False

In [37]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [38]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [39]:
history = model.fit(
      train_generator,
      steps_per_epoch=27,  
      epochs=2,
      validation_data=validation_generator,
      validation_steps=10,
      verbose=1)

Epoch 1/2
27/27 [==============================] - 17s 494ms/step - loss: 0.8607 - accuracy: 0.5272 - val_loss: 0.6505 - val_accuracy: 0.6513
Epoch 2/2
27/27 [==============================] - 12s 458ms/step - loss: 0.6142 - accuracy: 0.6735 - val_loss: 0.5446 - val_accuracy: 0.6769


In [40]:
import numpy as np
import tensorflow as tf
from skimage import color, io
from skimage.transform import resize

In [41]:
file_extension = "*.jpg"

train_forest_ic = io.imread_collection(os.path.join(train_forest_dir, file_extension))
train_mountain_ic = io.imread_collection(os.path.join(train_mountain_dir, file_extension))
validation_forest_ic = io.imread_collection(os.path.join(validation_forest_dir, file_extension))
validation_mountain_ic = io.imread_collection(os.path.join(validation_mountain_dir, file_extension))

In [42]:

y_forest_train = np.zeros(len(train_forest_ic.files))
y_mountain_train = np.zeros(len(train_mountain_ic.files))
y_forest_validation = np.ones(len(validation_forest_ic.files))
y_mountain_validation = np.ones(len(validation_mountain_ic.files))

print(y_forest_train.shape, y_mountain_train.shape, y_forest_validation.shape, y_mountain_validation.shape)

(268,) (252,) (60,) (122,)


In [43]:
train_forest_ic = [resize(img, (224, 224)) for img in train_forest_ic]
train_mountain_ic = [resize(img, (224, 224)) for img in train_mountain_ic]
validation_forest_ic = [resize(img, (224, 224)) for img in validation_forest_ic]
validation_mountain_ic = [resize(img, (224, 224)) for img in validation_mountain_ic]

In [44]:
X_train, y_train = np.concatenate((train_forest_ic, train_mountain_ic)), np.concatenate((y_forest_train, y_mountain_train))
X_validation, y_validation = np.concatenate((validation_forest_ic, validation_mountain_ic)), np.concatenate((y_forest_validation, y_mountain_validation))

### Instatiate Model

In [45]:
resnet = ResNet50(weights='imagenet', include_top=False)

In [46]:
for layer in resnet.layers:
    layer.trainable = False

### Fit Model

In [47]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [48]:
model.fit(
    X_train, y_train,
    epochs=15,
    verbose=1)

Epoch 1/15
17/17 [==============================] - 11s 430ms/step - loss: 0.0155 - accuracy: 1.0000
Epoch 2/15
17/17 [==============================] - 7s 435ms/step - loss: 1.5221e-11 - accuracy: 1.0000
Epoch 3/15
17/17 [==============================] - 8s 446ms/step - loss: 2.1772e-12 - accuracy: 1.0000
Epoch 4/15
17/17 [==============================] - 7s 433ms/step - loss: 1.5443e-12 - accuracy: 1.0000
Epoch 5/15
17/17 [==============================] - 8s 455ms/step - loss: 1.4516e-12 - accuracy: 1.0000
Epoch 6/15
17/17 [==============================] - 8s 447ms/step - loss: 1.4348e-12 - accuracy: 1.0000
Epoch 7/15
17/17 [==============================] - 8s 446ms/step - loss: 1.4317e-12 - accuracy: 1.0000
Epoch 8/15
17/17 [==============================] - 8s 461ms/step - loss: 1.4312e-12 - accuracy: 1.0000
Epoch 9/15
17/17 [==============================] - 8s 460ms/step - loss: 1.4311e-12 - accuracy: 1.0000
Epoch 10/15
17/17 [==============================] - 8s 461ms/step 

In [49]:
model.evaluate(X_validation, y_validation)

6/6 [==============================] - 4s 432ms/step - loss: 27.5236 - accuracy: 0.0000e+00


[27.52364730834961, 0.0]

## Part 2 : Custom CNN Model




이 단계에서는 Keras를 사용하여 자신 만의 CNN을 작성하고 훈련합니다. <br/>
네트워크에 적어도 하나의 Conv 레이어와 pooling 레이어가있는 아키텍처를 만들어 사용해 보세요. <br/> 아래는 여러분이 참고할 수 있도록 표시한 결과이며 여러분의 마음대로 설계하여도 됩니다.

### Make a Custom Model

In [50]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [51]:
model_scratch = Sequential()
model_scratch.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model_scratch.add(MaxPooling2D(2,2))
model_scratch.add(Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model_scratch.add(MaxPooling2D(2,2))
model_scratch.add(Flatten())
model_scratch.add(Dense(128, activation='relu'))
model_scratch.add(Dense(1, activation='sigmoid'))

In [52]:
model_scratch.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 200704)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               25690240  
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

### Compile Model

In [53]:
# Compile Model
model_scratch.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fit Model




In [54]:
# Fit Model
model_scratch.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_validation, y_validation))

Epoch 1/5
17/17 [==============================] - 7s 401ms/step - loss: 0.0531 - accuracy: 0.9385 - val_loss: 112.8856 - val_accuracy: 0.0000e+00
Epoch 2/5
17/17 [==============================] - 7s 390ms/step - loss: 6.8911e-26 - accuracy: 1.0000 - val_loss: 112.8856 - val_accuracy: 0.0000e+00
Epoch 3/5
17/17 [==============================] - 7s 388ms/step - loss: 6.8911e-26 - accuracy: 1.0000 - val_loss: 112.8856 - val_accuracy: 0.0000e+00
Epoch 4/5
17/17 [==============================] - 7s 398ms/step - loss: 6.8911e-26 - accuracy: 1.0000 - val_loss: 112.8856 - val_accuracy: 0.0000e+00
Epoch 5/5
17/17 [==============================] - 7s 391ms/step - loss: 6.8911e-26 - accuracy: 1.0000 - val_loss: 112.8856 - val_accuracy: 0.0000e+00
